In [ ]:
### From transcript via chunking to summary. I incorrectly used different prompts to retain as much information, I didnt think it would work out and it cost a lot of time. If I were to do this over, I would standardize my prompts across all analysis and summarization functions ###

%pip show openai

# Install the OpenAI package
%pip install openai

# Setup OpenAI Client
client = OpenAI(
  api_key=os.environ.get('OPENAI_API_KEY', '')
)


import openai
import os

# Import the new OpenAI client class
from openai import OpenAI

# Setup OpenAI Client
client = OpenAI(
  api_key=os.environ.get('OPENAI_API_KEY', '')
)

# Simple test to verify the setup
def test_openai_api():
    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": "Say this is a test"}
            ],
            temperature=0  # Set temperature to 0 for deterministic results
        )
        print(response.choices[0].message.content.strip())
    except Exception as e:
        print(f"An error occurred: {e}")

# Run the test
test_openai_api()

# Function to read the transcript with different encoding
def read_transcript(file_path):
    with open(file_path, 'r', encoding='iso-8859-1') as file:
        transcript = file.read()
    return transcript

# Use the full path to the transcript file
file_path = '/Users/eliecervalencia/Documents/Transcriptie Voedselbank Focusgroep/Focusgroep Voedselbank Transcriptie.txt'
transcript = read_transcript(file_path)

# Print the first 500 characters to verify the content
print(transcript[:500])

print("Transcript read successfully.")

# Function to split transcript based on word ranges
def split_by_word_ranges(text, word_ranges):
    words = text.split()
    chunks = [' '.join(words[start:end]) for start, end in word_ranges]
    return chunks

# Define word ranges based on exercises
exercise_word_ranges = [
    (0, 1644),
    (1644, 3691),
    (3691, 7153),  # corrected the range to avoid overlap
    (7153, 10676),
    (10676, 13249),
    (13249, 18546)
]

# Split the transcript into chunks based on word ranges
transcript_chunks = split_by_word_ranges(transcript, exercise_word_ranges)

# Print the first chunk to verify
print(transcript_chunks[0])
print("Transcript split into exercise chunks successfully.")

# Write each chunk to a separate file
output_dir = '/Users/eliecervalencia/Documents/Transcriptie Voedselbank Focusgroep/chunks'
os.makedirs(output_dir, exist_ok=True)

for i, chunk in enumerate(transcript_chunks):
    with open(os.path.join(output_dir, f'chunk_{i+1}.txt'), 'w', encoding='utf-8') as file:
        file.write(chunk)

print(f"Chunks written to files in {output_dir} successfully.")

# Function to split text into smaller sub-chunks if it exceeds the token limit
def split_into_subchunks(text, max_tokens=8192):
    words = text.split()
    subchunks = []
    current_chunk = []
    current_length = 0
    
    for word in words:
        current_length += len(word) + 1  # +1 for the space
        if current_length > max_tokens:
            subchunks.append(' '.join(current_chunk))
            current_chunk = [word]
            current_length = len(word) + 1
        else:
            current_chunk.append(word)
    
    if current_chunk:
        subchunks.append(' '.join(current_chunk))
    
    return subchunks

# Function to summarize each chunk
def summarize_chunk(chunk):
    subchunks = split_into_subchunks(chunk, max_tokens=1500)  # use a smaller token limit for safety
    summaries = []
    for subchunk in subchunks:
        try:
            response = client.chat.completions.create(
                model="gpt-4",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": f"Please summarize the following text:\n\n{subchunk}"}
                ],
                temperature=0  # Set temperature to 0 for deterministic results
            )
            summary = response.choices[0].message.content.strip()
            summaries.append(summary)
        except Exception as e:
            print(f"An error occurred while summarizing: {e}")
            return None
    return ' '.join(summaries)

# Summarize each chunk and save the summaries
summaries = []
for i, chunk in enumerate(transcript_chunks):
    summary = summarize_chunk(chunk)
    if summary:
        summaries.append(summary)
        print(f"Summary for chunk {i+1}:\n{summary}\n")

# Write summaries to a file
summaries_file_path = os.path.join(output_dir, 'summaries.txt')
with open(summaries_file_path, 'w', encoding='utf-8') as file:
    for i, summary in enumerate(summaries):
        file.write(f"Summary for chunk {i+1}:\n{summary}\n\n")

print(f"Summaries written to {summaries_file_path} successfully.")

##

# Function to analyze well-being topics in each chunk
def analyze_chunk(chunk):
    subchunks = split_into_subchunks(chunk, max_tokens=1500)  # use a smaller token limit for safety
    topics_responses = []
    interrelations_responses = []
    for subchunk in subchunks:
        try:
            # Identify well-being topics
            topics_response = client.chat.completions.create(
                model="gpt-4",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": f"Identify and list the topics discussed in the following text that participants believe impact their well-being, either positively or negatively. Provide quotes to support each topic identified. Consider any aspect of their lives that they mention as influencing their quality of life, well-being, or overall life satisfaction:\n\n{subchunk}"}
                ],
                temperature=0
            )
            topics_responses.append(topics_response.choices[0].message.content.strip())
            
            # Explain interrelations of well-being topics
            interrelations_response = client.chat.completions.create(
                model="gpt-4",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": f"Explain how the identified topics related to well-being are interrelated in the following text. Provide quotes or specific examples from the text to support your explanation. Focus on how these topics impact participants' well-being and their experiences with the informal foodbank:\n\n{subchunk}"}
                ],
                temperature=0
            )
            interrelations_responses.append(interrelations_response.choices[0].message.content.strip())
        except Exception as e:
            print(f"An error occurred while analyzing: {e}")
            return None, None
    
    return ' '.join(topics_responses), ' '.join(interrelations_responses)

# Analyze each chunk and save the results
analysis_results = []
for i, chunk in enumerate(transcript_chunks):
    topics, interrelations = analyze_chunk(chunk)
    if topics and interrelations:
        analysis_results.append({
            "chunk": i + 1,
            "topics": topics,
            "interrelations": interrelations
        })
        print(f"Analysis for chunk {i+1}:\nTopics:\n{topics}\nInterrelations:\n{interrelations}\n")

# Write the partial analyses to separate files
for result in analysis_results:
    partial_analysis_file_path = os.path.join(output_dir, f'partial_analysis_chunk_{result["chunk"]}.txt')
    with open(partial_analysis_file_path, 'w', encoding='utf-8') as file:
        file.write(f"Analysis for chunk {result['chunk']}:\nTopics:\n{result['topics']}\nInterrelations:\n{result['interrelations']}\n\n")

print("Partial analyses written to separate files successfully.")

# Combine all partial analyses into one document
combined_analysis_file_path = os.path.join(output_dir, 'combined_analysis.txt')
with open(combined_analysis_file_path, 'w', encoding='utf-8') as combined_file:
    for result in analysis_results:
        combined_file.write(f"Analysis for chunk {result['chunk']}:\nTopics:\n{result['topics']}\nInterrelations:\n{result['interrelations']}\n\n")

print(f"Combined analysis written to {combined_analysis_file_path} successfully.")


###


# Function to combine and analyze all partial analyses into one encapsulating analysis
def combine_and_analyze_analyses(partial_analyses):
    combined_analysis_text = "\n\n".join(partial_analyses)
    combined_analysis_chunks = split_into_subchunks(combined_analysis_text, max_tokens=5000)  # Split into smaller chunks
    combined_analysis_responses = []

    for chunk in combined_analysis_chunks:
        try:
            response = client.chat.completions.create(
                model="gpt-4",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": f"Analyze the following combined analyses and provide a comprehensive analysis that encapsulates the key insights, themes, and interrelations identified in each partial analysis:\n\n{chunk}"}
                ],
                temperature=0
            )
            combined_analysis_responses.append(response.choices[0].message.content.strip())
        except Exception as e:
            print(f"An error occurred while combining and analyzing the analyses: {e}")
            return None

    combined_analysis = ' '.join(combined_analysis_responses)
    return combined_analysis

# Function to split text into smaller sub-chunks if it exceeds the token limit
def split_into_subchunks(text, max_tokens=8192):
    words = text.split()
    subchunks = []
    current_chunk = []
    current_length = 0
    
    for word in words:
        current_length += len(word) + 1  # +1 for the space
        if current_length > max_tokens:
            subchunks.append(' '.join(current_chunk))
            current_chunk = [word]
            current_length = len(word) + 1
        else:
            current_chunk.append(word)
    
    if current_chunk:
        subchunks.append(' '.join(current_chunk))
    
    return subchunks

# Read the partial analyses from the files
partial_analyses = []
for i in range(1, 7):
    partial_analysis_file_path = os.path.join(output_dir, f'partial_analysis_chunk_{i}.txt')
    with open(partial_analysis_file_path, 'r', encoding='utf-8') as file:
        partial_analyses.append(file.read())

# Combine and analyze the partial analyses
combined_analysis = combine_and_analyze_analyses(partial_analyses)

# Write the combined analysis to a file
if combined_analysis:
    combined_analysis_file_path = os.path.join(output_dir, 'combined_analysis_encapsulated.txt')
    with open(combined_analysis_file_path, 'w', encoding='utf-8') as file:
        file.write(combined_analysis)

    print(f"Combined analysis written to {combined_analysis_file_path} successfully.")
else:
    print("Failed to produce a combined analysis.")

##

import os
from openai import OpenAI

# Setup OpenAI Client
client = OpenAI(
    api_key=''
)

# Test the setup with a simple call
def test_openai_api():
    try:
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": "Say this is a test",
                }
            ],
            model="gpt-4",
        )
        print(chat_completion.choices[0].message.content.strip())
    except Exception as e:
        print(f"An error occurred: {e}")

# Run the test
test_openai_api()

# Read and split the combined analysis
combined_analysis_file_path = '/Users/eliecervalencia/Documents/Transcriptie Voedselbank Focusgroep/chunks/combined_analysis.txt'
with open(combined_analysis_file_path, 'r', encoding='utf-8') as file:
    combined_analysis_text = file.read()

# Function to split text into smaller sub-chunks if it exceeds the token limit
def split_into_subchunks(text, max_tokens=2000):
    words = text.split()
    subchunks = []
    current_chunk = []
    current_length = 0

    for word in words:
        current_length += len(word) + 1  # +1 for the space
        if current_length > max_tokens:
            subchunks.append(' '.join(current_chunk))
            current_chunk = [word]
            current_length = len(word) + 1
        else:
            current_chunk.append(word)

    if current_chunk:
        subchunks.append(' '.join(current_chunk))

    return subchunks

# Function to summarize the combined analysis
def summarize_combined_analysis(combined_text):
    subchunks = split_into_subchunks(combined_text, max_tokens=2000)
    condensed_parts = []
    for subchunk in subchunks:
        try:
            response = client.chat.completions.create(
                model="gpt-4",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": f"Condense the following text into a summary that fits within one word page:\n\n{subchunk}"}
                ],
                temperature=0
            )
            condensed_part = response.choices[0].message.content.strip()
            condensed_parts.append(condensed_part)
        except Exception as e:
            print(f"An error occurred while condensing the summary: {e}")
            return None
    final_combined_summary = ' '.join(condensed_parts)
    return final_combined_summary

# Condense the combined analysis
condensed_summary = summarize_combined_analysis(combined_analysis_text)
if condensed_summary:
    condensed_summary_file_path = os.path.join(output_dir, 'condensed_combined_analysis.txt')
    with open(condensed_summary_file_path, 'w', encoding='utf-8') as file:
        file.write(condensed_summary)
    print(f"Condensed combined analysis written to {condensed_summary_file_path} successfully.")
else:
    print("Failed to produce condensed summary.")


##

import os
from openai import OpenAI

# Setup OpenAI Client
client = OpenAI(
    api_key=os.environ.get('OPENAI_API_KEY', '')
)

# Path to the condensed combined analysis file
condensed_summary_file_path = ''
/Users/eliecervalencia/Documents/Transcriptie Voedselbank Focusgroep/chunks/condensed_combined_analysis.txt
# Function to read the condensed combined analysis file
def read_condensed_analysis(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        condensed_analysis_text = file.read()
    return condensed_analysis_text

# Read the condensed analysis text
condensed_analysis_text = read_condensed_analysis(condensed_summary_file_path)



##

import os

# Function to read the condensed combined analysis file
def read_condensed_analysis(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        condensed_analysis_text = file.read()
    return condensed_analysis_text

# Path to the condensed combined analysis file
condensed_summary_file_path = '/Users/eliecervalencia/Documents/Transcriptie Voedselbank Focusgroep/chunks/condensed_combined_analysis.txt'

# Read the condensed analysis text
condensed_analysis_text = read_condensed_analysis(condensed_summary_file_path)

# Print the first 1000 characters to verify
print(condensed_analysis_text[:1000])


##

#### Enter: Prompt 1 #### (Bad analysis I know, should have used the same prompt from the beginning but I did not manage, too many errors, too unfamiliar with python, I was too messy) ###

from openai import OpenAI
import tiktoken

# Setup OpenAI Client
client = OpenAI(
    api_key=os.getenv('OPENAI_API_KEY', '')
)

# Function to count tokens using tiktoken
def count_tokens(text, model="gpt-4o"):
    enc = tiktoken.encoding_for_model(model)
    tokens = enc.encode(text)
    return len(tokens)

# Function to split text into smaller sub-chunks if it exceeds the token limit
def split_into_subchunks(text, max_tokens=1000):
    words = text.split()
    subchunks = []
    current_chunk = []
    current_length = 0

    for word in words:
        current_length += len(word) + 1  # +1 for the space
        if current_length > max_tokens:
            subchunks.append(' '.join(current_chunk))
            current_chunk = [word]
            current_length = len(word) + 1
        else:
            current_chunk.append(word)

    if current_chunk:
        subchunks.append(' '.join(current_chunk))

    return subchunks

# Split the condensed analysis text into smaller chunks
subchunks = split_into_subchunks(condensed_analysis_text, max_tokens=1000)
print(f"Number of subchunks created: {len(subchunks)}")

# Function to summarize the text with the matrix development prompt
def summarize_text(text):
    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": f"Please summarize the following text with a focus on identifying indicator pairs related to well-being. Consider factors such as frequency of co-occurrence, emotional context, and moments of agreement or disagreement:\n\n{text}"}
            ],
            temperature=0
        )
        summary = response.choices[0].message.content.strip()
        return summary
    except Exception as e:
        print(f"An error occurred while summarizing the text: {e}")
        return None

# Summarize each subchunk
summaries = []
for chunk in subchunks:
    summary = summarize_text(chunk)
    if summary:
        summaries.append(summary)
    else:
        break  # Stop if there's an error to avoid unnecessary requests

# Combine all summaries into one
combined_summary = ' '.join(summaries)

# Save the combined summary to the specified file
output_file_path = "/Users/eliecervalencia/Documents/Transcriptie Voedselbank Focusgroep/chunks/combined_summary.txt"
with open(output_file_path, 'w') as file:
    file.write(combined_summary)

# Print the combined summary to verify
combined_summary_length = count_tokens(combined_summary)
print(f"Length of combined summary: {combined_summary_length} tokens")
print(f"Combined Summary:\n{combined_summary[:1000]}")  # Print the first 1000 characters of the summary


##### Enter: Prompt 2 Matrix development ####

# Read the combined summary from the specified file
combined_summary_path = "/Users/eliecervalencia/Documents/Transcriptie Voedselbank Focusgroep/chunks/combined_summary.txt"
with open(combined_summary_path, 'r') as file:
    combined_summary = file.read()

# Function to generate the symmetric matrix using GPT-4-turbo
def generate_symmetric_matrix(text):
    try:
        response = client.chat.completions.create(
            model="gpt-4-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": f"Create a symmetric matrix of indicator pairs related to well-being from the following text. Include all unique indicators as both rows and columns. For each cell, provide the tie strength from 0 to 5, considering factors such as frequency of co-occurrence, emotional context, and moments of agreement or disagreement. Assign a tie strength of 0 for the weakest links where the indicators have minimal or no connection, and replace all 1's with 0's. Justify the tie strength for each pair.\n\n{text}"}
            ],
            temperature=0
        )
        response_message = response.choices[0].message.content.strip()
        return response_message
    except Exception as e:
        print(f"An error occurred while generating the symmetric matrix: {e}")
        return None

# Generate the symmetric matrix
symmetric_matrix_text = generate_symmetric_matrix(combined_summary)

# Print the generated symmetric matrix text to verify
print(symmetric_matrix_text)

# Save the symmetric matrix to a txt file
output_txt_file_path = "/Users/eliecervalencia/Documents/Transcriptie Voedselbank Focusgroep/chunks/symmetric_matrix.txt"
with open(output_txt_file_path, 'w') as file:
    file.write(symmetric_matrix_text)

# Convert the symmetric matrix text to a CSV format and save it
import pandas as pd
from io import StringIO

# Assume the symmetric matrix is formatted as a markdown table in the response text
matrix_df = pd.read_csv(StringIO(symmetric_matrix_text), sep="|", skipinitialspace=True)

# Clean up the column headers (strip any leading/trailing spaces)
matrix_df.columns = matrix_df.columns.str.strip()

# Save the dataframe to a CSV file
output_csv_file_path = "/Users/eliecervalencia/Documents/Transcriptie Voedselbank Focusgroep/chunks/symmetric_matrix.csv"
matrix_df.to_csv(output_csv_file_path, index=False)


#### Network visualization ####

import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

# Create the DataFrame from the provided matrix
data = {
    'Indicator': ['Financial Struggles', 'Food Bank Assistance', 'Unemployment', 'Cost of Living', 
                  'Community Support', 'Dependence on Food Bank Services', 'Social Support', 'Access to Food', 
                  'Mental Health', 'Physical Health', 'Energy Consumption', 'Reluctance to Use Gas', 
                  'National Issues', 'Perceived Neglect of Local Problems', 'Housing', 'Future Prospects'],
    'Financial Struggles': [0, 5, 4, 5, 2, 5, 3, 4, 4, 3, 2, 2, 3, 4, 4, 3],
    'Food Bank Assistance': [5, 0, 3, 4, 5, 5, 5, 5, 3, 3, 2, 2, 2, 3, 3, 2],
    'Unemployment': [4, 3, 0, 5, 2, 4, 2, 3, 3, 2, 1, 1, 2, 3, 3, 4],
    'Cost of Living': [5, 4, 5, 0, 2, 4, 2, 4, 3, 2, 4, 4, 3, 4, 4, 3],
    'Community Support': [2, 5, 2, 2, 0, 5, 5, 4, 2, 2, 1, 1, 2, 3, 2, 2],
    'Dependence on Food Bank Services': [5, 5, 4, 4, 5, 0, 5, 5, 3, 3, 2, 2, 2, 3, 3, 2],
    'Social Support': [3, 5, 2, 2, 5, 5, 0, 5, 4, 3, 1, 1, 2, 3, 2, 2],
    'Access to Food': [4, 5, 3, 4, 4, 5, 5, 0, 3, 3, 2, 2, 2, 3, 3, 2],
    'Mental Health': [4, 3, 3, 3, 2, 3, 4, 3, 0, 5, 2, 2, 2, 3, 3, 3],
    'Physical Health': [3, 3, 2, 2, 2, 3, 3, 3, 5, 0, 2, 2, 2, 3, 3, 2],
    'Energy Consumption': [2, 2, 1, 4, 1, 2, 1, 2, 2, 2, 0, 5, 2, 3, 2, 1],
    'Reluctance to Use Gas': [2, 2, 1, 4, 1, 2, 1, 2, 2, 2, 5, 0, 2, 3, 2, 1],
    'National Issues': [3, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 0, 5, 3, 2],
    'Perceived Neglect of Local Problems': [4, 3, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 5, 0, 4, 3],
    'Housing': [4, 3, 3, 4, 2, 3, 2, 3, 3, 3, 2, 2, 3, 4, 0, 4],
    'Future Prospects': [3, 2, 4, 3, 2, 2, 2, 2, 3, 2, 1, 1, 2, 3, 4, 0]
}

df = pd.DataFrame(data).set_index('Indicator')

# Create the graph
G = nx.Graph()

# Add nodes
for indicator in df.index:
    G.add_node(indicator)

# Add edges
for i, indicator in enumerate(df.index):
    for j, value in enumerate(df.iloc[i]):
        if value > 0:
            G.add_edge(indicator, df.columns[j], weight=value)

# Draw the graph
plt.figure(figsize=(12, 12))
pos = nx.spring_layout(G, seed=42)  # positions for all nodes

# Draw nodes and labels
nx.draw_networkx_nodes(G, pos, node_size=7000, node_color='skyblue', alpha=0.7)
nx.draw_networkx_labels(G, pos, font_size=10)

# Draw edges
edges = nx.draw_networkx_edges(G, pos, width=1.0, alpha=0.5)
nx.draw_networkx_edge_labels(G, pos, edge_labels={(u, v): f'{d["weight"]}' for u, v, d in G.edges(data=True)}, font_color='red')

# Show the plot
plt.title('Well-being Indicator Network')
plt.show()

#

%pip install networkx
%pip install matplotlib
%pip install pandas
%pip install scipy

#

import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

# Create the DataFrame from the provided matrix
data = {
    'Indicator': ['Financial Struggles', 'Food Bank Assistance', 'Unemployment', 'Cost of Living', 
                  'Community Support', 'Dependence on Food Bank Services', 'Social Support', 'Access to Food', 
                  'Mental Health', 'Physical Health', 'Energy Consumption', 'Reluctance to Use Gas', 
                  'National Issues', 'Perceived Neglect of Local Problems', 'Housing', 'Future Prospects'],
    'Financial Struggles': [0, 5, 4, 5, 2, 5, 3, 4, 4, 3, 2, 2, 3, 4, 4, 3],
    'Food Bank Assistance': [5, 0, 3, 4, 5, 5, 5, 5, 3, 3, 2, 2, 2, 3, 3, 2],
    'Unemployment': [4, 3, 0, 5, 2, 4, 2, 3, 3, 2, 1, 1, 2, 3, 3, 4],
    'Cost of Living': [5, 4, 5, 0, 2, 4, 2, 4, 3, 2, 4, 4, 3, 4, 4, 3],
    'Community Support': [2, 5, 2, 2, 0, 5, 5, 4, 2, 2, 1, 1, 2, 3, 2, 2],
    'Dependence on Food Bank Services': [5, 5, 4, 4, 5, 0, 5, 5, 3, 3, 2, 2, 2, 3, 3, 2],
    'Social Support': [3, 5, 2, 2, 5, 5, 0, 5, 4, 3, 1, 1, 2, 3, 2, 2],
    'Access to Food': [4, 5, 3, 4, 4, 5, 5, 0, 3, 3, 2, 2, 2, 3, 3, 2],
    'Mental Health': [4, 3, 3, 3, 2, 3, 4, 3, 0, 5, 2, 2, 2, 3, 3, 3],
    'Physical Health': [3, 3, 2, 2, 2, 3, 3, 3, 5, 0, 2, 2, 2, 3, 3, 2],
    'Energy Consumption': [2, 2, 1, 4, 1, 2, 1, 2, 2, 2, 0, 5, 2, 3, 2, 1],
    'Reluctance to Use Gas': [2, 2, 1, 4, 1, 2, 1, 2, 2, 2, 5, 0, 2, 3, 2, 1],
    'National Issues': [3, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 0, 5, 3, 2],
    'Perceived Neglect of Local Problems': [4, 3, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 5, 0, 4, 3],
    'Housing': [4, 3, 3, 4, 2, 3, 2, 3, 3, 3, 2, 2, 3, 4, 0, 4],
    'Future Prospects': [3, 2, 4, 3, 2, 2, 2, 2, 3, 2, 1, 1, 2, 3, 4, 0]
}

df = pd.DataFrame(data).set_index('Indicator')

# Create the graph
G = nx.Graph()

# Add nodes
for indicator in df.index:
    G.add_node(indicator)

# Add edges
for i, indicator in enumerate(df.index):
    for j, value in enumerate(df.iloc[i]):
        if value > 0:
            G.add_edge(indicator, df.columns[j], weight=value)

# Draw the graph using Kamada-Kawai layout
plt.figure(figsize=(12, 12))
pos = nx.kamada_kawai_layout(G)  # Kamada-Kawai layout

# Draw nodes and labels
nx.draw_networkx_nodes(G, pos, node_size=7000, node_color='skyblue', alpha=0.7)
nx.draw_networkx_labels(G, pos, font_size=10)

# Draw edges
edges = nx.draw_networkx_edges(G, pos, width=1.0, alpha=0.5)
nx.draw_networkx_edge_labels(G, pos, edge_labels={(u, v): f'{d["weight"]}' for u, v, d in G.edges(data=True)}, font_color='red')

# Show the plot
plt.title('Well-being Indicator Network (Kamada-Kawai Layout)')
plt.show()

#

import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.cm as cm
import numpy as np
from matplotlib.patches import FancyArrowPatch

# Assuming df is already created from the previous steps

# Create the graph
G = nx.Graph()

# Add nodes
for indicator in df.index:
    G.add_node(indicator)

# Add edges
for i, indicator in enumerate(df.index):
    for j, value in enumerate(df.iloc[i]):
        if value > 0:
            G.add_edge(indicator, df.columns[j], weight=value)

# Calculate node sizes based on the degree (number of edges) with a scaling factor
node_degrees = [G.degree(n) for n in G.nodes()]
min_degree = min(node_degrees)
max_degree = max(node_degrees)
scaling_factor = 200  # Adjust as needed for visual clarity

node_sizes = [scaling_factor * (G.degree(n) - min_degree + 1) for n in G.nodes()]

# Calculate edge colors and transparency based on weight
edge_weights = [d['weight'] for _, _, d in G.edges(data=True)]
cmap = cm.get_cmap('Blues')
norm = mcolors.Normalize(vmin=min(edge_weights), vmax=max(edge_weights))
edge_colors = [cmap(norm(w)) for w in edge_weights]
edge_alphas = [norm(w) for w in edge_weights]

# Draw the graph using Kamada-Kawai layout
plt.figure(figsize=(12, 12))
pos = nx.kamada_kawai_layout(G)  # Kamada-Kawai layout

# Draw nodes and labels
nx.draw_networkx_nodes(G, pos, node_size=node_sizes, node_color='skyblue', alpha=0.7)
nx.draw_networkx_labels(G, pos, font_size=10)

# Draw edges with varying thickness based on weight, colors, and transparency, and make them less curved
ax = plt.gca()
for (u, v, d), color, alpha in zip(G.edges(data=True), edge_colors, edge_alphas):
    edge = FancyArrowPatch(
        posA=pos[u], posB=pos[v],
        connectionstyle='arc3,rad=0.1',  # Reduced curvature
        arrowstyle='-',  # No arrow heads
        color=color, alpha=alpha,
        linewidth=d['weight']
    )
    ax.add_patch(edge)

# Show the plot
plt.title('Well-being Indicator Network (Kamada-Kawai Layout)')
plt.show()

#


